In [1]:
from imp import reload

from hpcp_loader_for_softdtw import *
from torch.utils.data import DataLoader
import models.BaseSPPNet as models
# reload(models)
from config import DefaultConfig, opt
from tqdm import tqdm
import torch
from utility import *
import matplotlib.pyplot as plt
import json
import torch.nn.functional as F
from torch.nn.parallel.data_parallel import DataParallel
from torch.nn import *
import os
import pandas as pd
import seaborn as sns
import resource
import librosa
import numpy as np

## rlimit = resource.getrlimit(resource.RLIMIT_NOFILE)
## torch.backends.cudnn.benchmark = True  # cudnn有很多种并行计算卷积的算法，
## resource.setrlimit(resource.RLIMIT_NOFILE, (2048, rlimit[1]))

os.environ["CUDA_VISIBLE_DEVICES"] = "4"
device_ids = [4]

In [2]:
def neuralwarp_train(kwargs):
    # 多尺度图片训练 396+

    print(kwargs)

    with open(kwargs['params']) as f:
        params = json.load(f)
    if kwargs['manner'] == 'train':
        params['is_train'] = True
    else:
        params['is_train'] = False
    params['batch_size'] = kwargs['batch_size']
    if torch.cuda.device_count() > 1:
        print("-------------------Parallel_GPU_Train--------------------------")
        parallel = True
    else:
        print("------------------Single_GPU_Train----------------------")
        parallel = False
    opt.feature = 'cqt'
    opt.notes = 'SoftDTW'
    opt.model = 'SoftDTW'
    opt.batch_size = 'batch_size'
    opt._parse(kwargs) # opt.model = NeuralDTW_CNN_Mask_dilation_SPP6
    model = getattr(models, opt.model)(params)

    p = 'check_points/' + model.model_name + opt.notes
    if kwargs['model'] == 'NeuralDTW_CNN_Mask_dilation_SPP':
        f = os.path.join(p, "0704_19:58:25.pth")
    elif kwargs['model'] == 'NeuralDTW_CNN_Mask_dilation_SPP2':
        f = os.path.join(p, "0806_10:01:20.pth")
    elif kwargs['model'] == 'NeuralDTW_CNN_Mask_dilation':
        f = os.path.join(p, "0704_06:40:41.pth")
    elif kwargs['model'] == 'NeuralDTW_CNN_Mask_dilation_SPP5':
        f = os.path.join(p, '0819_12:54:27.pth')
    elif kwargs['model'] == 'NeuralDTW_CNN_Mask_dilation_SPP6':
        f = os.path.join(p, '0819_01:03:39.pth')
    elif kwargs['model'] == 'NeuralDTW_2':
        f = os.path.join(p, '0907_09:58:52.pth')
    elif kwargs['model'] == 'NeuralDTW_seq_vector2':
        f = os.path.join(p, '0921_09:40:49.pth')
    opt.load_model_path = f
    if kwargs['model'] != 'NeuralDTW' and kwargs['manner'] != 'train':
        if opt.load_latest is True:
            model.load_latest(opt.notes)
        elif opt.load_model_path:
            print("load_model:", opt.load_model_path)
            model.load(opt.load_model_path)

    if parallel == True:
        model = DataParallel(model)
    model.to(opt.device)
    torch.multiprocessing.set_sharing_strategy('file_system')
    
    # step2: data
    out_length = 400
    if kwargs['model'] == 'NeuralDTW_CNN_Mask_300':
        out_length = 300
    if kwargs['model'] == 'NeuralDTW_CNN_Mask_spp':
        train_data0 = triplet_CQT(out_length=200, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
        train_data1 = triplet_CQT(out_length=300, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
        train_data2 = triplet_CQT(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
    else:
        train_data0 = triplet_CQT(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
        train_data1 = triplet_CQT(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
        train_data2 = triplet_CQT(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
#     val_data80 = CQT('songs80', out_length=kwargs['test_length'])
#     val_data = CQT('songs350', out_length=kwargs['test_length'])
    # val_data80 =CQT_cut_test('songs80', out_length=None)
    # val_data = CQT_cut_test('songs350', out_length=None)
    # val_data_marukars = CQT('Mazurkas',out_length=kwargs['test_length'])
    # val_song2000 = CQT('test',out_length=kwargs['test_length'])
    train_dataloader0 = DataLoader(train_data0, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
    train_dataloader1 = DataLoader(train_data1, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
    train_dataloader2 = DataLoader(train_data2, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
#     val_dataloader80 = DataLoader(val_data80, 1, shuffle=False, num_workers=1)
#     val_dataloader = DataLoader(val_data, 1, shuffle=False, num_workers=1)
    # val_dataloader_marukars = DataLoader(val_data_marukars,300, shuffle=False, num_workers=1)
    # val_dataloader2000 = DataLoader(val_song2000,300,shuffle=False,num_workers=16)
    if kwargs['manner'] == 'test':

        val_data80_1 = CQT('songs80', out_length=kwargs['test_length'])
        val_data_1 = CQT('songs350', out_length=kwargs['test_length'])
        val_dataloader80_1 = DataLoader(val_data80_1, 1, shuffle=False, num_workers=1)
        val_dataloader_1 = DataLoader(val_data_1, 1, shuffle=False, num_workers=1)
        map1 = val_quick(model, val_dataloader_1)
        map2 = val_quick(model, val_dataloader80_1)
        print(map1)
        print(map2)
    elif kwargs['manner'] == 'visualize':
        # visualize(model, val_dataloader80)
        # test_visualize(model)
        vis(model, val_dataloader80)
    elif kwargs['manner'] == 'mul_test':
        p = 'check_points/' + model.model_name + opt.notes
        l = sorted(os.listdir(p))[: 20]
        best_MAP, MAP = 0, 0
        for f in l:
            f = os.path.join(p, f)
            model.load(f)
            model.to(opt.device)
            MAP += val_slow_batch(model, val_dataloader, batch=400, is_dis=kwargs['zo'])
            MAP += val_slow_batch(model, val_dataloader80, batch=400, is_dis=kwargs['zo'])
            if MAP > best_MAP:
                print('--best result--')
                best_MAP = MAP
            MAP = 0
    elif kwargs['manner'] == "test_covers80":
        test_covers80(model, val_dataloader80)
    elif kwargs['manner'] == 'tempo_change_test':
        val_data08 = CQT_test('tempo08', out_length=kwargs['test_length'])
        val_data09 = CQT_test('tempo09', out_length=kwargs['test_length'])
        val_data11 = CQT_test('tempo11', out_length=kwargs['test_length'])
        val_data12 = CQT_test('tempo12', out_length=kwargs['test_length'])
        val_dataloader08 = DataLoader(val_data08, 100, shuffle=False, num_workers=1)
        val_dataloader09 = DataLoader(val_data09, 100, shuffle=False, num_workers=1)
        val_dataloader11 = DataLoader(val_data11, 100, shuffle=False, num_workers=1)
        val_dataloader12 = DataLoader(val_data12, 100, shuffle=False, num_workers=1)
        print("tempo_change_08:")
        val_slow_batch_test(model, val_dataloader08, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("tempo_change_09:")
        val_slow_batch_test(model, val_dataloader09, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("tempo_change_11:")
        val_slow_batch_test(model, val_dataloader11, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("tempo_change_12:")
        val_slow_batch_test(model, val_dataloader12, val_dataloader80, batch=100, is_dis=kwargs['zo'])
    elif kwargs['manner'] == 'key_change_test':
        val_data6 = CQT_test('key_6', out_length=kwargs['test_length'])
        val_datan6 = CQT_test('key_n6', out_length=kwargs['test_length'])
        val_data12 = CQT_test('key_12', out_length=kwargs['test_length'])
        val_datan12 = CQT_test('key_n12', out_length=kwargs['test_length'])
        val_dataloader6 = DataLoader(val_data6, 100, shuffle=False, num_workers=1)
        val_dataloadern6 = DataLoader(val_datan6, 100, shuffle=False, num_workers=1)
        val_dataloader12 = DataLoader(val_data12, 100, shuffle=False, num_workers=1)
        val_dataloadern12 = DataLoader(val_datan12, 100, shuffle=False, num_workers=1)
        print("key_6:")
        val_slow_batch_test(model, val_dataloader6, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("key_n6:")
        val_slow_batch_test(model, val_dataloadern6, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("key_12:")
        val_slow_batch_test(model, val_dataloader12, val_dataloader80, batch=100, is_dis=kwargs['zo'])
        print("key_n12:")
        val_slow_batch_test(model, val_dataloadern12, val_dataloader80, batch=100, is_dis=kwargs['zo'])
    else: # 'train'
        # step3: criterion and optimizer
        be = torch.nn.BCELoss()

        lr = opt.lr
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=opt.weight_decay)

        # if parallel is True:
        #     optimizer = torch.optim.Adam(model.module.parameters(), lr=lr, weight_decay=opt.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=10,
                                                               verbose=True, min_lr=5e-6)
        # step4: train
        best_MAP = 0
        for epoch in range(opt.max_epoch):
            print(f'---------epoch {epoch}----------')
            running_loss = 0
            num = 0
            for ii, ((a0, p0, n0, la0, lp0, ln0), (a1, p1, n1, la1, lp1, ln1), (a2, p2, n2, la2, lp2, ln2)) in tqdm(
                    enumerate(zip(train_dataloader0, train_dataloader1, train_dataloader2))):
                # for ii, (a2, p2, n2) in tqdm(enumerate(train_dataloader2)):
                for flag in range(3):
                    if flag == 0:
                        a, p, n, la, lp, ln = a0, p0, n0, la0, lp0, ln0
                    elif flag == 1:
                        a, p, n, la, lp, ln = a1, p1, n1, la1, lp1, ln1
                    else:
                        a, p, n, la, lp, ln = a2, p2, n2, la2, lp2, ln2
                    B, _, _, _ = a.shape
                    if kwargs["zo"] == True:
                        target = torch.cat((torch.zeros(B), torch.ones(B))).cuda()
                    else:
                        target = torch.cat((torch.ones(B), torch.zeros(B))).cuda()
                    # train model
                    a = a.requires_grad_().to(opt.device)
                    p = p.requires_grad_().to(opt.device)
                    n = n.requires_grad_().to(opt.device)

                    optimizer.zero_grad()
                    pred = model(a, p, n)
                    pred = pred.squeeze(1)
                    print(pred, target
                         )
                    loss = be(pred, target)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    num += a.shape[0]

                if ii % 1 == 0 :
                    print(f'num={num}')
                    running_loss /= num
                    print("train_loss:", running_loss)

#                     MAP = 0
#                     print("Youtube350:")
#                     MAP += val_slow_batch(model, val_dataloader, batch=100, is_dis=kwargs['zo'])
#                     print("CoverSong80:")
#                     MAP += val_slow_batch(model, val_dataloader80, batch=100, is_dis=kwargs['zo'])
                    # print("Youtube350:")
                    # MAP += val_quick(model, val_dataloader)
                    # print("CoverSong80:")
                    # MAP += val_quick(model, val_dataloader80)
                    # print("Marukars:")
                    # MAP += val_slow_batch(model, val_dataloader_marukars, batch=100, is_dis=kwargs['zo'])
#                     if MAP > best_MAP:
#                         best_MAP = MAP
#                         print('*****************BEST*****************')
#                     if kwargs['save_model'] == True:
#                         if parallel:
#                             model.module.save(opt.notes)
#                         else:
#                             model.save(opt.notes)
                    scheduler.step(running_loss)
                    running_loss = 0
                    num = 0

In [ ]:
kwargs = {'model': 'NeuralDTW_CNN_Mask_dilation_SPP6', 'num_workers': 16, 'batch_size': 5, 'is_label': True,
              'is_random': False, 'notes': 'mask', 'save_model': True, 'manner': 'train',
              'params': 'params/neuraldtw/mask0', 'test_length': 400, 'zo': False }
neuralwarp_train(kwargs)

In [ ]:
train_data0 = triplet_CQT(out_length=400, is_label=True)
train_dataloader0 = DataLoader(train_data0, 5, shuffle=True)

for a, p, n, la, lp, ln in train_dataloader0:
    break

opt.model = 'NeuralDTW_CNN_Mask_dilation_SPP6'

model = getattr(models, opt.model)(None)
model.to('cuda')
pass

a = a.to('cuda')
p = p.to('cuda')
n = n.to('cuda')

In [ ]:
@torch.no_grad()
def val_quick(softdtw, dataloader):
    # softdtw: model
    softdtw.eval()
    softdtw.model.eval()
    labels = []
    temp = []
    count = -1
    for ii, (data, label) in tqdm(enumerate(dataloader)):
        labels.append(label)
    labels = torch.cat(labels, dim=0)
    N = labels.shape[0] # 样本数
    dis2d = np.zeros((N, N))
    for ii, (data, label) in tqdm(enumerate(dataloader)):
        data = data.cuda()
        print(data.shape)
        count += 1
        if count == 0:
            temp.append((data, count))
        else:
            for i in range(len(temp)):
                dis = softdtw.multi_compute_s(data, temp[i][0]).data.cpu().numpy()
                dis2d[temp[i][1]][count], dis2d[count][temp[i][1]] = -dis, -dis
            temp.append((data, count))

    MAP, top10, rank1 = calc_MAP(dis2d[0:labels.shape[0], 0:labels.shape[0]], labels)
    print(MAP, top10, rank1)
    softdtw.train()
    softdtw.model.train()
    return MAP

In [ ]:
def neuralwarp_train_1(**kwargs):
    # 多尺度图片训练 396+
    # print(kwargs)
    # print("Mask == 1")

    print(kwargs)

    with open(kwargs['params']) as f:
        params = json.load(f)
    if kwargs['manner'] == 'train':
        params['is_train'] = True
    else:
        params['is_train'] = False
    params['batch_size'] = kwargs['batch_size']
    if torch.cuda.device_count() > 1:
        print("-------------------Parallel_GPU_Train--------------------------")
        parallel = True
    else:
        print("------------------Single_GPU_Train----------------------")
        parallel = False
    opt.feature = 'cqt'
    opt.notes = 'SoftDTW'
    opt.model = 'SoftDTW'
    opt.batch_size = 'batch_size'
    opt._parse(kwargs)
    model = getattr(models, opt.model)(params)

    p = 'check_points/' + model.model_name + opt.notes
    f = os.path.join(p,
                     "0620_07:05:30.pth")  # 使用Neural_dtw目前最优 0620_07:05:30.pth cover80 map:0.705113267654046 0.08125 7.96875

    opt.load_model_path = f
    if kwargs['model'] != 'NeuralDTW' and kwargs['manner'] != 'train':
        if opt.load_latest is True:
            model.load_latest(opt.notes)
        elif opt.load_model_path:
            print("load_model:", opt.load_model_path)
            model.load(opt.load_model_path)

    if parallel == True:
        model = DataParallel(model)
    model.to(opt.device)
    torch.multiprocessing.set_sharing_strategy('file_system')
    # step2: data
    out_length = 400
    train_data0 = triplet_CQT_test(out_length=300, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
    train_data1 = triplet_CQT_test(out_length=400, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
    train_data2 = triplet_CQT_test(out_length=500, is_label=kwargs['is_label'], is_random=kwargs['is_random'])
    val_data80 = CQT_cut_test('songs80', out_length=None)
    val_data = CQT_cut_test('songs350', out_length=None)
    train_dataloader0 = DataLoader(train_data0, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
    train_dataloader1 = DataLoader(train_data1, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
    train_dataloader2 = DataLoader(train_data2, opt.batch_size, shuffle=True, num_workers=opt.num_workers)
    val_dataloader80 = DataLoader(val_data80, 1, shuffle=False, num_workers=1)
    val_dataloader = DataLoader(val_data, 1, shuffle=False, num_workers=1)
    if kwargs['manner'] == 'test':

        val_data80_1 = CQT_cut_test('songs80', out_length=kwargs['test_length'])
        val_data_1 = CQT_cut_test('songs350', out_length=kwargs['test_length'])
        val_dataloader80_1 = DataLoader(val_data80_1, 1, shuffle=False, num_workers=1)
        val_dataloader_1 = DataLoader(val_data_1, 1, shuffle=False, num_workers=1)
        val_quick(model, val_dataloader_1)
        val_quick(model, val_dataloader80_1)

    elif kwargs['manner'] == 'visualize':
        # visualize(model, val_dataloader80)
        test_visualize(model)
    else:
        # step3: criterion and optimizer
        be = torch.nn.BCELoss()

        lr = opt.lr
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=opt.weight_decay)

        # if parallel is True:
        #     optimizer = torch.optim.Adam(model.module.parameters(), lr=lr, weight_decay=opt.weight_decay)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.8, patience=10,
                                                               verbose=True, min_lr=5e-6)
        # step4: train
        best_MAP = 0
        for epoch in range(opt.max_epoch):
            running_loss = 0
            num = 0
            for ii, ((a0, p0, n0, la0, lp0, ln0), (a1, p1, n1, la1, lp1, ln1), (a2, p2, n2, la2, lp2, ln2)) in tqdm(
                    enumerate(zip(train_dataloader0, train_dataloader1, train_dataloader2))):
                # for ii, (a2, p2, n2) in tqdm(enumerate(train_dataloader2)):
                for flag in range(3):
                    if flag == 0:
                        a, p, n, la, lp, ln = a0, p0, n0, la0, lp0, ln0
                    elif flag == 1:
                        a, p, n, la, lp, ln = a1, p1, n1, la1, lp1, ln1
                    else:
                        a, p, n, la, lp, ln = a2, p2, n2, la2, lp2, ln2
                    B, _, _, _ = a.shape
                    if kwargs["zo"] == True:
                        target = torch.cat((torch.zeros(B), torch.ones(B))).cuda()
                    else:
                        target = torch.cat((torch.ones(B), torch.zeros(B))).cuda()
                    # train model
                    a = a.requires_grad_().to(opt.device)
                    p = p.requires_grad_().to(opt.device)
                    n = n.requires_grad_().to(opt.device)

                    optimizer.zero_grad()
                    pred = model(a, p, n)
                    pred = pred.squeeze(1)
                    loss = be(pred, target)
                    loss.backward()
                    optimizer.step()

                    running_loss += loss.item()
                    num += a.shape[0]

                if ii % 5000 == 0 and ii != 0:
                    running_loss /= num
                    print("train_loss:", running_loss)

#                     MAP = 0
                    # print("Youtube350:")
                    # MAP += val_slow_batch(model, val_dataloader, batch=100, is_dis=kwargs['zo'])
                    # print("CoverSong80:")
                    # MAP += val_slow_batch(model, val_dataloader80, batch=100, is_dis=kwargs['zo'])
#                     print("Youtube350:")
#                     MAP += val_quick(model, val_dataloader)
#                     print("CoverSong80:")
#                     MAP += val_quick(model, val_dataloader80)
                    # print("Marukars:")
                    # MAP += val_slow_batch(model, val_dataloader_marukars, batch=100, is_dis=kwargs['zo'])
#                     if MAP > best_MAP:
#                         best_MAP = MAP
#                         print('*****************BEST*****************')
#                     if kwargs['save_model'] == True:
#                         if parallel:
#                             model.module.save(opt.notes)
#                         else:
#                             model.save(opt.notes)
                    scheduler.step(running_loss)
                    running_loss = 0
                    num = 0

In [ ]:
@torch.no_grad()
def val_slow(softdtw, dataloader, style='null'):
    softdtw.eval()
    softdtw.model.eval()

    seqs, labels = [], []
    for ii, (data, label) in tqdm(enumerate(dataloader)):
        input = data.cuda()

        seqs.append(input)
        labels.append(label)
    labels = torch.cat(labels, dim=0)

    N = labels.shape[0]
    if N == 350:
        query_l = [i // 100 for i in range(100 * 100, 350 * 100)]
        ref_l = [i for i in range(100)] * 250
    else:
        query_l = [i // N for i in range(N * N)]
        ref_l = [i for i in range(N)] * N
    dis2d = np.zeros((N, N))

    for st in range(0, N * N if N != 350 else 100 * 250):
        query = seqs[query_l[st]]
        ref = seqs[ref_l[st]]
        if style == 'min':
            T = min(query.shape[1], ref.shape[1])
            query, ref = query[:, :T, :], ref[:, :T, :]
        # print(softdtw.metric(query, ref))
        s = softdtw.multi_compute_s(query, ref).data.cpu().numpy()
        i, j = query_l[st], ref_l[st]
        dis2d[i, j], dis2d[j, i] = -s[0], -s[0]

    if len(labels) == 350:
        MAP, top10, rank1 = calc_MAP(dis2d, labels, [100, 350])
    else:
        MAP, top10, rank1 = calc_MAP(dis2d, labels)
    print(MAP, top10, rank1)

    softdtw.train()
    softdtw.model.train()
    return MAP

In [ ]:
@torch.no_grad()
def val_slow_batch(softdtw, dataloader, batch=200, is_dis='False'):
    softdtw.eval()
    if torch.cuda.device_count() > 1:
        softdtw.module.model.eval()
    else:
        softdtw.model.eval()
    seqs, labels = [], []
    for ii, (data, label) in tqdm(enumerate(dataloader)):
        input = data.cuda()
        # _, seq, _ = softdtw.model(input)
        seqs.append(input)
        labels.append(label)
    seqs = torch.cat(seqs, dim=0)
    labels = torch.cat(labels, dim=0)

    N = labels.shape[0]
    if N == 350:
        query_l = [i // 100 for i in range(100 * 100, 350 * 100)]
        ref_l = [i for i in range(100)] * 250
    else:
        query_l = [i // N for i in range(N * N)]
        ref_l = [i for i in range(N)] * N
    dis2d = np.zeros((N, N))

    N = N * N if N != 350 else 100 * 250
    for st in range(0, N, batch):
        fi = (st + batch) if st + batch <= N else N
        query = seqs[query_l[st: fi], :, :]
        ref = seqs[ref_l[st: fi], :, :]
        if torch.cuda.device_count() > 1:
            s = softdtw.module.multi_compute_s(query, ref).data.cpu().numpy()
        else:
            s = softdtw.multi_compute_s(query, ref).data.cpu().numpy()
        for k in range(st, fi):
            i, j = query_l[k], ref_l[k]
            # print(i, j)
            if is_dis:
                dis2d[i, j] = s[k - st]
            else:
                dis2d[i, j] = -s[k - st]

    if len(labels) == 350:
        MAP, top10, rank1 = calc_MAP(dis2d, labels, [100, 350])
    else:
        MAP, top10, rank1 = calc_MAP(dis2d, labels)
    print(MAP, top10, rank1)

    softdtw.train()
    if torch.cuda.device_count() > 1:
        softdtw.module.model.train()
    else:
        softdtw.model.train()
    return MAP

In [ ]:
@torch.no_grad()
def val_slow_batch_test(softdtw, dataloaderQ, dataloaderR, batch=200, is_dis='False'):
    softdtw.eval()
    if torch.cuda.device_count() > 1:
        softdtw.module.model.eval()
    else:
        softdtw.model.eval()
    seqsQ, seqs, labels = [], [], []

    for ii, (data, label) in tqdm(enumerate(dataloaderQ)):
        input = data.cuda()
        # _, seq, _ = softdtw.model(input)
        seqsQ.append(input)

        labels.append(label)
    for ii, (data, label) in tqdm(enumerate(dataloaderR)):
        input = data.cuda()
        # _, seq, _ = softdtw.model(input)
        seqs.append(input)
    seqsQ = torch.cat(seqsQ, dim=0)
    seqs = torch.cat(seqs, dim=0)
    labels = torch.cat(labels, dim=0)

    N = labels.shape[0]
    if N == 350:
        query_l = [i // 100 for i in range(100 * 100, 350 * 100)]
        ref_l = [i for i in range(100)] * 250
    else:
        query_l = [i // N for i in range(N * N)]
        ref_l = [i for i in range(N)] * N
    dis2d = np.zeros((N, N))

    N = N * N if N != 350 else 100 * 250
    for st in range(0, N, batch):
        fi = (st + batch) if st + batch <= N else N
        query = seqsQ[query_l[st: fi], :, :]
        ref = seqs[ref_l[st: fi], :, :]
        if torch.cuda.device_count() > 1:
            s = softdtw.module.multi_compute_s(query, ref).data.cpu().numpy()
        else:
            s = softdtw.multi_compute_s(query, ref).data.cpu().numpy()
        for k in range(st, fi):
            i, j = query_l[k], ref_l[k]
            # print(i, j)
            if is_dis:
                dis2d[i, j] = s[k - st]
            else:
                dis2d[i, j] = -s[k - st]

    if len(labels) == 350:
        MAP, top10, rank1 = calc_MAP(dis2d, labels, [100, 350])
    else:
        MAP, top10, rank1 = calc_MAP(dis2d, labels)
    print(MAP, top10, rank1)

    softdtw.train()
    if torch.cuda.device_count() > 1:
        softdtw.module.model.train()
    else:
        softdtw.model.train()
    return MAP

In [ ]:
if __name__ == '__main__':
    import fire

    fire.Fire()